In [6]:
# pip install transformers torch


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/15/fc/7b6dd7e1adc0a6407b845ed4be1999e98b6917d0694e57316d140cc85484/transformers-4.39.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/134.8 kB ? eta -:--:--
     -------- ---------------------------- 30.7/134.8 kB 445.2 kB/s eta 0:00:01
     -------------------------------------- 134.8/134.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/5c/01/5ab75f138bf32d7a69df61e4997e24eccad87cc009f5fb7e2a31af8a4036/torch-2.2.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/8b/69/acdf492db27dea7be5c63053230130e0574fd8a376de3555d5f8bbc3d3ad/filelock-3.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from htt

In [7]:
sentences = [
    # Positive
    "The new café downtown has the most delightful coffee and pastries.",
    "She received a promotion at work, which was a significant achievement for her career.",
    "His presentation at the conference was met with great enthusiasm and applause.",
    "The sunset at the beach was breathtakingly beautiful, making it a perfect evening.",
    # Neutral
    "The library will be open from 9 AM to 5 PM during the weekdays.",
    "The cat sat on the windowsill, observing the birds outside.",
    "The car was parked in the driveway next to a tall oak tree.",
    # Negative
    "The weather turned stormy and ruined the outdoor picnic plans.",
    "He missed his flight due to a delay in the traffic, causing a lot of stress.",
    "The meal was unsatisfactory, with most dishes being too salty to enjoy."
]


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sentences)

print("TF-IDF Feature Matrix:\n", X_tfidf.toarray())


TF-IDF Feature Matrix:
 [[0.         0.         0.24885515 0.         0.         0.
  0.         0.         0.         0.         0.         0.33460439
  0.         0.         0.         0.         0.33460439 0.
  0.         0.33460439 0.         0.33460439 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.33460439 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.2844443  0.33460439 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.33460439 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.27100164 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.30281131 0.         0.         0.         0.2252097  0.
  0.         0.         0.      

In [9]:
import gensim.downloader as api
from gensim.models import Word2Vec
import numpy as np

# Load a pre-trained Word2Vec model
word2vec_model = api.load("glove-wiki-gigaword-100")  # Using GloVe model for simplicity

def get_word2vec_embedding(sentence, model):
    words = sentence.lower().split()
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

word2vec_features = np.array([get_word2vec_embedding(sentence, word2vec_model) for sentence in sentences])
print("Word2Vec Feature Matrix:\n", word2vec_features)


Word2Vec Feature Matrix:
 [[-2.14623995e-02  1.09984994e-01  2.04161078e-01 -1.73558906e-01
   2.46310309e-01  2.71314204e-01 -4.38794047e-02  3.88246983e-01
  -1.14405498e-01  6.79877847e-02 -1.70494001e-02 -3.32056999e-01
   1.10827886e-01 -1.78224929e-02 -3.06769945e-02 -9.55059566e-03
   2.59837985e-01 -1.57097101e-01 -2.18238994e-01  7.84843937e-02
   2.94771373e-01  5.96240051e-02  2.69671977e-01 -1.64658099e-01
   3.88144791e-01 -2.24369243e-01  2.88484953e-02 -4.40923035e-01
  -2.29097322e-01 -1.63288191e-01 -1.22581497e-01  4.38793570e-01
   6.98937997e-02  2.29461625e-01 -1.79128982e-02  4.12480593e-01
   1.72096804e-01  2.14577317e-01  6.74661994e-02 -3.64933372e-01
  -1.89994991e-01 -3.24995995e-01  1.29918009e-01 -3.02268595e-01
   3.99811000e-01  3.39243501e-01  1.79949000e-01 -2.33767986e-01
   1.86154991e-01 -3.08474720e-01 -2.05572516e-01 -1.16264500e-01
   1.74757950e-02  7.00110316e-01 -4.56001669e-01 -2.20500088e+00
  -1.54117301e-01 -7.75995925e-02  1.33851933e+00 

In [10]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Use the pooler_output for sentence representation
    return outputs.pooler_output.detach().numpy()

bert_features = np.array([get_bert_embedding(sentence, tokenizer, model) for sentence in sentences])
print("BERT Feature Matrix:\n", bert_features)


c:\Users\Owner\Desktop\AIML\FoundationMachineLearningFramework\CSCN8010-Foundations-of-Machine-Learning\venv\CSCN8010_classic_ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Owner\Desktop\AIML\FoundationMachineLearningFramework\CSCN8010-Foundations-of-Machine-Learning\venv\CSCN8010_classic_ml\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs

BERT Feature Matrix:
 [[[-0.84032536 -0.43594855 -0.859679   ... -0.37501222 -0.7484938
    0.912589  ]]

 [[-0.79361784 -0.25934976 -0.5767554  ... -0.11025967 -0.56743383
    0.77622586]]

 [[-0.734331   -0.30408517 -0.6168387  ... -0.39061344 -0.4974861
    0.5957187 ]]

 ...

 [[-0.90824795 -0.50547343 -0.9329736  ... -0.7774874  -0.74270964
    0.9557329 ]]

 [[-0.7386516  -0.41390106 -0.7658367  ... -0.47223532 -0.57514673
    0.81724954]]

 [[-0.8442355  -0.45612717 -0.9098576  ... -0.7645484  -0.66478693
    0.9206641 ]]]


In [12]:
import numpy as np

# Assigning numerical values to the sentiment: Positive = 1, Neutral = 0, Negative = -1
y = np.array([1, 1, 1, 1,  # Positive sentences
              0, 0, 0,    # Neutral sentences
              -1, -1, -1])  # Negative sentences

y


array([ 1,  1,  1,  1,  0,  0,  0, -1, -1, -1])

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Assuming X_tfidf is your TF-IDF feature matrix and y is your target variable
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_predictions


array([0, 1])

In [15]:
y_test

array([-1,  1])

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Assuming X_glove or X_bert is your feature matrix from GloVe or BERT
X_train, X_test, y_train, y_test = train_test_split(word2vec_features, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')  # Linear kernel is often a good choice for text data
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_predictions

array([1, 1])

In [21]:
from tensorflow.keras.utils import to_categorical

# Map the target variable from -1, 0, 1 to 0, 1, 2 for one-hot encoding
y_mapped = y + 1  # Now, -1 becomes 0, 0 becomes 1, and 1 becomes 2

# One-hot encode the mapped target variable
y_one_hot = to_categorical(y_mapped, num_classes=3)


In [22]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

# Assuming bert_features is your feature matrix
X_train, X_test, y_train, y_test = train_test_split(bert_features, y_one_hot, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train.shape[1:])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')  # Change to 3 units with softmax activation
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1) - 1  # Subtract 1 to map back to -1, 0, 1


Epoch 1/10
1/1 [==============================] - 0s 377ms/step - loss: 1.7253 - accuracy: 0.1667 - val_loss: 2.2609 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 23ms/step - loss: 0.9284 - accuracy: 0.6667 - val_loss: 1.9258 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 1.2003 - accuracy: 0.3333 - val_loss: 1.5297 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 23ms/step - loss: 1.4746 - accuracy: 0.1667 - val_loss: 1.1459 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 24ms/step - loss: 1.2283 - accuracy: 0.5000 - val_loss: 1.0910 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 23ms/step - loss: 0.7554 - accuracy: 0.6667 - val_loss: 1.0666 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 21ms/step - loss: 1.3245 - accuracy: 0.5000 - val_loss: 1.0761 - val_accuracy: 0.5000
Epoch 8/10

In [23]:
predicted_classes

array([0, 0], dtype=int64)